In [ ]:
import numpy as np
import pandas as pd

print(np.__version__)
print(pd.__version__)


In [ ]:
import pandas as pd

df = pd.read_csv("../data/depop_vintage_womens_tees.csv")
df.head()


In [ ]:
import os
os.getcwd()


In [ ]:
df = pd.read_csv("data/depop_vintage_womens_tees.csv")
df.head()


In [ ]:
import pandas as pd

df = pd.read_csv("data/depop_vintage_womens_tees.csv")
df.head()

In [ ]:
import os

os.listdir()


In [ ]:
import os
os.listdir("data")


In [ ]:
df = pd.read_csv("data/depop_vintage_womens_tees.csv")
df.head()


In [ ]:
df["condition"] = df["condition"].str.lower()
df["brand"] = df["brand"].str.lower()


In [ ]:
df.head()


In [ ]:
df.info()


In [ ]:
df.describe()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.scatterplot(data=df, x="price", y="likes")
plt.title("Likes vs Price")
plt.show()


Engagement generally decreases as price increases, though some higher-priced listings still receive strong engagement, suggesting brand and condition influence visibility beyond price alone.

In [ ]:
sns.boxplot(data=df, x="condition", y="likes")
plt.title("Likes by Condition")
plt.show()


Listings in better condition categories tend to receive higher engagement, though overlap exists across conditions.

In [ ]:
top_brands = df["brand"].value_counts().head(10).index

sns.boxplot(
    data=df[df["brand"].isin(top_brands)],
    x="brand",
    y="likes"
)
plt.xticks(rotation=45)
plt.title("Likes by Brand (Top 10)")
plt.show()


Certain brands consistently outperform others in engagement, even at similar price points.

In [ ]:
median_likes = df["likes"].median()
df["high_engagement"] = (df["likes"] >= median_likes).astype(int)

df["high_engagement"].value_counts()


Listings were labeled as high or low engagement based on the median number of likes, enabling future classification modeling.

In [ ]:
import numpy as np

# 1) Standardize to string, strip spaces
df["likes_raw"] = df["likes"]  # keep original for transparency
df["likes"] = df["likes"].astype(str).str.strip()

# 2) Convert "99+" -> 99 (cap)
df["likes"] = df["likes"].str.replace("+", "", regex=False)

# 3) Convert to numeric; invalid -> NaN
df["likes"] = pd.to_numeric(df["likes"], errors="coerce")

# 4) Decide how to handle missing likes (recommended: fill with 0)
df["likes"] = df["likes"].fillna(0).astype(int)

df[["likes_raw", "likes"]].head(15)


In [ ]:
median_likes = df["likes"].median()
df["high_engagement"] = (df["likes"] >= median_likes).astype(int)

median_likes, df["high_engagement"].value_counts()


In [ ]:
(df["likes_raw"].astype(str).str.contains("99", na=False)).sum()


In [ ]:
df["likes"].describe()


In [ ]:
df["days_since_posted"] = pd.to_numeric(df["days_since_posted"], errors="coerce").fillna(df["days_since_posted"].median())
df["price"] = pd.to_numeric(df["price"], errors="coerce")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.linear_model import LinearRegression, LogisticRegression

X = df[["price", "condition", "brand", "days_since_posted"]]
y_reg = df["likes"]
y_clf = df["high_engagement"]

categorical = ["condition", "brand"]
numeric = ["price", "days_since_posted"]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical),
        ("num", "passthrough", numeric),
    ]
)


In [ ]:
df.shape


In [ ]:
# Regression: predict likes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

X = df[["price", "condition", "brand", "days_since_posted"]]
y = df["likes"]

categorical = ["condition", "brand"]
numeric = ["price", "days_since_posted"]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical),
        ("num", "passthrough", numeric),
    ]
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

reg_model = Pipeline(
    steps=[
        ("prep", preprocessor),
        ("model", LinearRegression()),
    ]
)

reg_model.fit(X_train, y_train)
preds = reg_model.predict(X_test)

mae = mean_absolute_error(y_test, preds)
mae


In [ ]:
X = df[["price", "condition", "brand", "days_since_posted"]]
X.isna().sum()


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

categorical = ["condition", "brand"]
numeric = ["price", "days_since_posted"]

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median"))
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical),
        ("num", numeric_transformer, numeric),
    ]
)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

X = df[["price", "condition", "brand", "days_since_posted"]]
y = df["likes"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

reg_model = Pipeline(
    steps=[
        ("prep", preprocessor),
        ("model", LinearRegression()),
    ]
)

reg_model.fit(X_train, y_train)
preds = reg_model.predict(X_test)
mae = mean_absolute_error(y_test, preds)
mae


In [ ]:
X = df[["price", "condition", "brand", "days_since_posted"]]
y_reg = df["likes"]

print("X NaNs:\n", X.isna().sum(), "\n")
print("y_reg NaNs:", y_reg.isna().sum())


In [ ]:
# Drop rows that are completely blank
df = df.dropna(how="all")

# Drop rows missing essential columns
df = df.dropna(subset=["price", "condition", "brand", "days_since_posted", "likes"])

df.shape


In [ ]:
X = df[["price", "condition", "brand", "days_since_posted"]]
y_reg = df["likes"]

print("X NaNs:\n", X.isna().sum(), "\n")
print("y_reg NaNs:", y_reg.isna().sum())


In [ ]:
median_likes = df["likes"].median()
df["high_engagement"] = (df["likes"] >= median_likes).astype(int)

median_likes, df["high_engagement"].value_counts()


In [ ]:
import pandas as pd
import numpy as np

# 1) remove completely blank rows (safe)
df = df.dropna(how="all")

# 2) standardize likes -> numeric
df["likes"] = df["likes"].astype(str).str.strip()
df["likes"] = df["likes"].str.replace("+", "", regex=False)   # "99+" -> "99"
df["likes"] = pd.to_numeric(df["likes"], errors="coerce")     # strings -> numbers
df = df.dropna(subset=["likes"])                               # drop any weird leftovers
df["likes"] = df["likes"].astype(int)

# 3) (optional but recommended) ensure numeric fields are numeric
df["price"] = pd.to_numeric(df["price"], errors="coerce")
df["days_since_posted"] = pd.to_numeric(df["days_since_posted"], errors="coerce")

# 4) drop rows missing essentials for modeling
df = df.dropna(subset=["price", "condition", "brand", "days_since_posted"])

df[["likes", "price", "days_since_posted"]].dtypes, df.shape


In [ ]:
median_likes = df["likes"].median()
df["high_engagement"] = (df["likes"] >= median_likes).astype(int)

median_likes, df["high_engagement"].value_counts()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

X = df[["price", "condition", "brand", "days_since_posted"]]
y = df["likes"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

reg_model = Pipeline(
    steps=[
        ("prep", preprocessor),
        ("model", LinearRegression()),
    ]
)

reg_model.fit(X_train, y_train)
preds = reg_model.predict(X_test)

mae = mean_absolute_error(y_test, preds)
mae


### Regression Results
The regression model predicts listing engagement (likes) with a mean absolute
error of approximately 23 likes. Given the long-tailed and noisy nature of
marketplace engagement data, this provides a reasonable baseline rather than
precise prediction.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

y = df["high_engagement"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

clf_model = Pipeline(
    steps=[
        ("prep", preprocessor),
        ("model", LogisticRegression(max_iter=1000)),
    ]
)

clf_model.fit(X_train, y_train)
preds = clf_model.predict(X_test)

acc = accuracy_score(y_test, preds)
baseline = y_test.mean()

acc, baseline


### Classification Results
The classifier outperformed a naive baseline, indicating that pricing and
listing metadata (price, brand, condition, and time since posting) contain
meaningful signal for predicting engagement.


In [ ]:
import pandas as pd

feature_names = clf_model.named_steps["prep"].get_feature_names_out()
coefs = clf_model.named_steps["model"].coef_[0]

importance = (
    pd.DataFrame({"feature": feature_names, "coef": coefs})
      .sort_values("coef", ascending=False)
)

importance.head(10)


### Feature Importance Insights
Brand recognition and item condition were the strongest predictors of
engagement. Listings associated with recognizable brands showed a higher
likelihood of strong engagement, while higher prices generally reduced
engagement probability.


In [ ]:
import pandas as pd
df = pd.read_csv("../data/depop_vintage_womens_tees.csv")
df.head()


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/depop_vintage_womens_tees.csv")

print("rows, cols:", df.shape)
df.head(10)


In [ ]:
summary = pd.DataFrame({
    "dtype": df.dtypes.astype(str),
    "missing": df.isna().sum(),
    "missing_pct": (df.isna().mean() * 100).round(2),
    "n_unique": df.nunique(dropna=True)
}).sort_values("missing_pct", ascending=False)

summary.head(25)


In [ ]:
# show likely engagement-related columns (likes, saves, comments, etc.)
candidates = [c for c in df.columns if any(k in c.lower() for k in ["like", "save", "comment", "engage", "view", "click"])]
candidates


In [ ]:
df.shape


## Next
- Identify the best engagement metric (likes / saves / comments).
- Clean numeric columns (price, likes) and handle outliers.
- Explore relationships: price vs engagement, keywords/brand vs engagement.


In [ ]:
df["likes"].describe()


In [ ]:
df["likes"].isna().sum(), (df["likes"] < 0).sum()


In [ ]:
df["likes"].dtype


In [ ]:
df["likes"].unique()[:20]


In [ ]:
df["likes"] = (
    df["likes"]
    .astype(str)          # ensure everything is string
    .str.strip()          # remove whitespace
    .replace({"": None, "—": None, "None": None})
)

df["likes"] = pd.to_numeric(df["likes"], errors="coerce")


In [ ]:
df["likes"].dtype


In [ ]:
df["likes"].isna().sum(), (df["likes"] < 0).sum()


In [ ]:
df["likes"].describe()


## Likes Column Cleaning
The `likes` field was originally stored as text.
Values were stripped, invalid entries coerced to missing, and converted to numeric
to support quantitative analysis.


In [ ]:
high_engagement_threshold = df["likes"].quantile(0.75)
high_engagement_threshold


## Engagement Definition
Listings with likes in the top 25% (≥ ~23 likes) are classified as **high engagement**.
This threshold balances sample size with meaningful interest.


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.hist(df["likes"], bins=30)
plt.title("Distribution of Likes")
plt.xlabel("Likes")
plt.ylabel("Count")
plt.show()

plt.figure()
plt.hist(df["likes"], bins=30)
plt.yscale("log")
plt.title("Distribution of Likes (Log Scale)")
plt.xlabel("Likes")
plt.ylabel("Count (log)")
plt.show()


In [ ]:
df.dtypes.sort_values()


In [ ]:
plt.figure()
plt.scatter(df["days_since_posted"], df["likes"], alpha=0.6)
plt.xlabel("Days Since Posted")
plt.ylabel("Likes")
plt.title("Likes vs Days Since Posted")
plt.show()


In [ ]:
plt.figure()
plt.scatter(df["days_since_posted"], df["likes"], alpha=0.6)
plt.yscale("log")
plt.xlabel("Days Since Posted")
plt.ylabel("Likes (log scale)")
plt.title("Likes vs Days Since Posted (Log Scale)")
plt.show()


## Time Effect
Engagement increases with time on platform, but the relationship is nonlinear.
Most listings receive the majority of engagement early, with diminishing returns over time.


In [ ]:
plt.figure()
plt.scatter(df["price"], df["likes"], alpha=0.6)
plt.xlabel("Price")
plt.ylabel("Likes")
plt.title("Likes vs Price")
plt.show()


In [ ]:
plt.figure()
plt.scatter(df["price"], df["likes"], alpha=0.6)
plt.yscale("log")
plt.xlabel("Price")
plt.ylabel("Likes (log scale)")
plt.title("Likes vs Price (Log Scale)")
plt.show()


In [ ]:
high_engagement_threshold = df["likes"].quantile(0.75)

df["high_engagement"] = df["likes"] >= high_engagement_threshold

df["high_engagement"].value_counts()


## High Engagement Label
Listings are labeled as **high engagement** if their number of likes falls
in the top 25% of the dataset (≥ 75th percentile).


## Price Sensitivity
Lower-priced items tend to receive higher engagement.
High-priced listings show greater variance, suggesting niche or brand-driven interest.


In [ ]:
engagement_by_condition = (
    df.groupby("condition")["high_engagement"]
    .mean()
    .sort_values(ascending=False)
)

engagement_by_condition


In [ ]:
plt.figure()
engagement_by_condition.plot(kind="bar")
plt.ylabel("Proportion High Engagement")
plt.title("High Engagement Rate by Condition")
plt.show()
